# Подписки

*Выручка на пользователя в подписочных сервисах.*

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

from collections import namedtuple

np.random.seed(7)

В А/Б-тестах подписочных сервисов основной интерес представляет выручка на пользователя. В оценке стоит учитывать ожидаемую выручку от продлений. Могут быть варианты, как ее моделировать.

Конверсия в подписку и выручка на пользователя от первой оплаты.
В оплату - мультиномиальное распределение по тарифам.
Средняя выручка на пользователя - стоимость * конверсию.

Конверсии в оплату в случае N опций:

$$
P(\mathcal{D} | \mathcal{H}) = Mult(n_0, \dots, n_N | p_0, \dots, p_N)
$$

$$
P(\mathcal{H}) = 
Dir \left( p_{0}, \dots, p_{N}; \alpha_{0}, \dots, \alpha_{N} \right) 
$$

$$
\begin{split}
P(\mathcal{H} | \mathcal{D}) 
& \propto Mult(n_0, \dots, n_N | p_0, \dots, p_N) Dir \left( p_{0}, \dots, p_{N}; \alpha_{0}, \dots, \alpha_{N} \right)
\\
& =
Dir \left( p_{0}, \dots, p_{N}; \alpha_{0} + n_0, \dots, \alpha_{N} + n_N \right)
\end{split}
$$

$$
P(p_i | \mathcal{D} ) = 
\int dp_0 \dots dp_{i-1}dp_{i+i} \dots dp_N P(\mathcal{H} | \mathcal{D}) 
=
Beta( p_i; \alpha_i + n_i, \sum_{k=0}^{N} (\alpha_k + n_k) - \alpha_i - n_i )
$$


Пусть есть тарифы:  
-неделя 500  
-месяц 1000  
-год 5000  

Есть конверсии в эти тарифы: $p_0, p_1, p_2$.  
Средние чеки: конверсия * цену.  

In [42]:
def initial_params_dir(N):
    return np.ones(N)

def posterior_params_dir_mult(data, initial_pars):
    #u, c = np.unique(data, return_counts=True)
    post_pars = np.copy(initial_pars)
    for k, v in enumerate(data):
        post_pars[k] = post_pars[k] + v
    return post_pars

def posterior_dist_dir(params):
    return stats.dirichlet(alpha=params)

def marginal_pi_dist_dir(i, params):
    return stats.beta(a=params[i], b=np.sum(params) - params[i])

def posterior_pi_mean_95pdi(i, params):
    p = marginal_pi_dist_dir(i, params)
    m = p.mean()
    lower = p.ppf(0.025)
    upper = p.ppf(0.975)
    return m, lower, upper

def posterior_ords_dir_rvs(params, nsamp):
    ords = np.empty(nsamp)
    d = posterior_dist_dir(params)
    probs = d.rvs(size=nsamp)
    for i, p in enumerate(probs):
        ordtype = np.argmax(stats.multinomial.rvs(n=1, p=p))
        ords[i] = prices[ordtype]
    return ords

def posterior_ords_mean_rvs(params, nsample):
    probs = stats.dirichlet.rvs(alpha=params, size=nsample)
    means = np.sum(prices * probs, axis=1)
    return means


nsample = 1000

Npars = 3
p0 = 0.3
p1 = 0.6
p2 = 0.1

probs = np.array([p0, p1, p2])
prices = np.array([500, 1000, 5000])


#exact_dist = stats.multinomial(n=3, p=(p0,p1,p2))
#data = exact_dist.rvs(nsample)
data = stats.multinomial.rvs(n=nsample, p=(p0,p1,p2))
p_data = data / nsample
print(data)
pars = initial_params_dir(Npars)
pars = posterior_params_dir_mult(data, pars)
pars
#post_samp = posterior_nords_dir_rvs(pars, 100000)
pi = [posterior_pi_mean_95pdi(i, pars) for i in range(Npars)]


exact_mean = np.sum(probs * prices)
npostsamp = 50000
o = posterior_ords_dir_rvs(pars, npostsamp)
display(o)
m = posterior_ords_mean_rvs(pars, npostsamp)
display(m)


x = np.arange(0, Npars)
fig = go.Figure()
fig.add_trace(go.Bar(x=x, y=[p0, p1, p2], name='Точные конверсии', 
                         marker_color='black', width=0.1))
fig.add_trace(go.Bar(x=x, y=p_data, name='Выборка',
                         marker_color='green', opacity=0.3, width=0.1))
fig.add_trace(go.Scatter(x=x, 
                         y=[p[0] for p in pi],
                         error_y=dict(type='data', symmetric=False, array=[p[2] - p[0] for p in pi], arrayminus=[p[0] - p[1] for p in pi]), 
                         name='$\mbox{Оценки } p_i$',
                         mode='markers',
                         line_color='red',
                         opacity=0.8))
fig.update_layout(title='Заказы на посетителя',
                  xaxis_title='$p$',
                  yaxis_title='Вероятность',
                  xaxis_range=[-0.2, Npars-1],
                  hovermode="x",
                  barmode="group",
                  bargap=0.8,
                  bargroupgap=1,
                  height=550)
fig.show()


x = np.arange(0, 5000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=[exact_mean, exact_mean], 
                         y=[0, 0.2],
                         name='Точное среднее', 
                         mode='lines', line_dash='dash',
                         line_color='black'))
fig.add_trace(go.Histogram(x=m, histnorm='probability', name='$E[n]$', 
                           marker_color='black', opacity=0.3, nbinsx=round(50)))
fig.update_layout(title='Средний чек',
                  xaxis_title='Средний чек',
                  yaxis_title='Вероятность',
                  xaxis_range=[1000, 2000],
                  hovermode="x",
                  barmode="group",
                  height=550)
fig.show()

[293 600 107]


array([ 500.,  500., 1000., ..., 1000.,  500.,  500.])

array([1252.03793555, 1258.03297657, 1273.23383884, ..., 1292.19860855,
       1242.38070211, 1321.7292201 ])

Продления.  


Конверсии в продление на каждом шаге убывают близко к геометрической прогрессии.  
Точнее - показатель геометрической прогрессии неравномерный.  
Больше всего отписок после первого периода.  
Дальше отток уменьшается.  
Во многих продуктах возврат выходит на плато - отток близок 0.  

In [41]:
s_per = np.arange(11)
s_per

b = 0.70
part_geom = [b**i for i in s_per]
nonuniformgeom = [1, 0.6, 0.4, 0.3, 0.2, 0.18, 0.15, 0.1, 0.1, 0.09, 0.08]
#b = 0.8
#geom_damp = [(b * i / (i+1))**i for i in s_per]
c = 0.07
b = 0.6
geom_asy = [c + (1-c)*b**i for i in s_per]

fig = go.Figure()
fig.add_trace(go.Scatter(x=s_per, y=part_geom, name='Geom'))
fig.add_trace(go.Scatter(x=s_per, y=nonuniformgeom, name='NonUniformGeom'))
#fig.add_trace(go.Scatter(x=s_per, y=geom_dump, name='GeomDamp'))
fig.add_trace(go.Scatter(x=s_per, y=geom_asy, name='GeomAsy'))
fig.update_layout(
    yaxis_range=[0, 1.2]
)
fig.show()

Продление - тоже конверсия.  
Бета-распределение на каждом шаге.   
Если данных нет, то равномерное. 
Либо ближе к предыдущему.

Обычно нет данных, чтобы точно моделировать продления на каждом шаге. 

Может быть удобно посмотреть на пользователей по-отдельности.

<center>
<img src="../figs/subscriptions.png" alt="subscriptions"  width="400"/>
<em>Продления. </em>
</center>

Геометрическое распределение?   
Сколько шагов, пока не отвалится.  
Вероятность разная у каждого пользователя.  
Первое приближение - для каждого пользователя вероятность отвала на каждом шаге одинакова.  
Для отдельного пользователя геометрическое распределение.  
По всем - сумма (или произведение?) геометрических распределений.  
Сумма геометрических - негативное биномиальное. Количество попыток до заданного числа успехов.   
https://en.wikipedia.org/wiki/Geometric_distribution

Негативное биномиальное?   
Сколько всего попыток до заданного числа успехов.  
"Успех" - когда пользователь отписывается.  
Число успехов - отписались все пользователи.  
Попытки - продления.  
Для каждого пользователя вероятность отвала на каждом шаге одинакова.  
Нереалистично, но сойдет как первое приближение.   

Если не хочется загадывать сильно в будущее, можно ограничить количество продлений.  
Для каждого пользователя выписать произведение коверсий в продление.  
Мультиномиальное распределение с вероятностями $(1-p)$, $p(1-p)$, $p^2(1-p)$, $\dots$, $p^n(1-p)$.

Негативное биномиальное.  
https://en.wikipedia.org/wiki/Negative_binomial_distribution  
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.nbinom.html  
k - число провалов.  
$\theta$ - вероятность успеха.  
r - число успехов.  

$$
\mbox{NegBinom}(k; r, \theta) = \binom{k+r-1}{k} (1-\theta)^k \theta^r
$$

Если считать все конверсии в продления одинаковыми, конверсию можно оценивать как бета-распределения.  
Одинаковая конверсия по всем шагам - это как усреднение конверсии по всем шагам.  
Число продлений / число всех возможностей продолжить.  
После оценки конверсии генерировать общее количество продлений негативным биномиальным.  

Пришло $N=1000$ пользователей.  
$n_0 = 100$ оформили подписку.  
$n_1=70$ продлили за 1 период.  
$n_2=50$ продлили за 2 период.  
Всего возможностей для продления: $N = n_0 + n_1$.   
Продлений $n_s = n_1 + n_2$.

Оценка конверсии в продление:

$$
P(\mathcal{H} | \mathcal{D}) \propto P(\mathcal{D} | \mathcal{H}) P(\mathcal{H})
$$

$$
P(\mathcal{D} | \mathcal{H}) = P(n_s, N | p) = \mbox{Binom}(n_s, N | p) = C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
$$

$$
P(\mathcal{H}) = P(p) = \mbox{Beta}(p; \alpha, \beta)
$$

$$
\begin{split}
P(\mathcal{H} | \mathcal{D}) & = P(p | n_s, N) 
= \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)
\end{split}
$$


Всего оплат продлений: число "провалов" до заданного числа "успехов".  

$$
P(k; r, \theta) = C_{k+r-1}^{k} (1-\theta)^k \theta^r
\\
E[k] = \frac{r (1-\theta)}{\theta}
$$

Отписка - это "успех".  
Отписаться должны все текущие активные $r \to n_2$.  
Часть продлений уже была $n_s = n_1 + n_2$. Нужно учесть в общем числе.  
Продление подписки - это "провал" $k \to n_s$.  
Конверсией должно быть конверсией в отписку, а не продление $\theta \to (1-p)$.  

$$
S \sim n_s + \mbox{NegBinom}(s, n_2 | 1-p) \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)
$$

Сумма продлений (c - цена подписки):
$$
c S = c n_s + \sum c s P(s) = c n_s + c E[s] = c n_s + c \frac{n_2 p}{(1-p)}
$$

In [37]:
def posterior_dist_nbinom_p(s, f, a_prior=1, b_prior=1):
    a = a_prior + s
    b = b_prior + f 
    return stats.beta(a=a, b=b)

def prolongation_dist(initial_users, p_prolong):
    nbinom_p = 1 - p_prolong
    return stats.nbinom(n=initial_users, p=nbinom_p)

N = 1000
s0 = 100
s1 = 70

nbinom_p_dist = posterior_dist_nbinom_p(s=s0, f=s1)
nbinom_p_dist

x = np.linspace(0, 1, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=nbinom_p_dist.pdf(x)))
fig.update_layout(
    title=f"p",
    xaxis_title='p',
    yaxis_title='Prob'
)
fig.show()

# x = np.arange(nbinom_p_dist.mean() + nbinom_p_dist.std()*5)
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=x, y=nbinom_p_dist.pmf(k=x), mode='markers+lines', name='Subs'))
# fig.update_layout(
#     title=f"Nbinom, ns={ns}, p={p}",
#     xaxis_title='N',
#     yaxis_title='Prob'
# )
# fig.show()

In [29]:
p = 0.5
ns = 50

x = np.arange(ns/p * 2)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=stats.nbinom.pmf(k=x, n=ns, p=p), mode='markers+lines', name='Fails'))
fig.add_trace(go.Scatter(x=x+ns, y=stats.nbinom.pmf(k=x, n=ns, p=p), mode='markers+lines', name='Total'))
fig.update_layout(
    title=f"Nbinom, ns={ns}, p={p}",
    xaxis_title='N',
    yaxis_title='Prob'
)
fig.show()

In [42]:
x = np.linspace(0, 1, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x=x, a=1, b=1)))
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x=x, a=1, b=2)))
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x=x, a=1, b=3)))
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x=x, a=1, b=4)))
fig.add_trace(go.Scatter(x=x, y=stats.beta.pdf(x=x, a=1, b=5)))
# fig.update_layout(
#     yaxis_range=[0, 1.5]
# )
fig.show()

В оплату - мультиномиальное распределение по тарифам.
Средняя выручка на пользователя - стоимость * конверсию.

Что делать с продлениями?
Лучше всего ждать продлений и смотреть по данным. 
Допустим эти данные есть. 
Как тогда?

Если данных нет, или есть за первый период.
Что тогда?

Вариант - задавать предельное значение в каждый след. период.
Пользователей точно не станет больше, чем было.

Можно немного уменьшать.

Т.е. что-то типа равномерного распределения от 0 до последнего периода.
Лучше не равномерное, а спадающее по краям. Но с плато на большей части интервала.

Отписки могут быть с фиксированной скоростью, например по 5\%.  
Это геометрическая прогрессия.  
Но обычно немного не так.

Первый месяц отписывается много, дальше доля отписок уменьшается.

В случае N опций

$$
\begin{split}
P(\mathcal{D} | \mathcal{H}) & = Mult(n_0, \dots, n_N | p_0, \dots, p_N) = \frac{(n_0 + \dots + n_N)!}{n_{0}! \dots n_{N}!} p_{0}^{n_{0}} \dots p_{N}^{n_{N}} 
\\
P(\mathcal{H}) & = 
Dir \left( p_{0}, \dots, p_{N}; \alpha_{0}, \dots, \alpha_{N} \right) = 
\dfrac{1}{B( \alpha_{0}, \dots, \alpha_{N} )} \prod_{i=0}^{N} p_{i}^{\alpha_{i}-1},
\qquad
\sum_{i=0}^{N} p_i = 1,
\qquad
p_i \in [0, 1], 
\qquad
B(\alpha_{0}, \dots, \alpha_{N}) = 
\frac{\prod \limits_{i=0}^{N} \Gamma( \alpha_{i} )}
{\Gamma \left( \sum \limits_{i=0}^{N} \alpha_{i} \right)}
\\
P(\mathcal{H} | \mathcal{D}) 
& \propto Mult(n_0, \dots, n_N | p_0, \dots, p_N) Dir \left( p_{0}, \dots, p_{N}; \alpha_{0}, \dots, \alpha_{N} \right)
\\
& \propto
p_{0}^{n_{0}} \dots p_{N}^{n_{N}} 
\prod _{i=0}^{N} p_{i}^{\alpha_{i}-1}
\\
& \propto
\prod_{i=0}^{N} p_{i}^{n_{i} + \alpha_{i} - 1}
\\
& =
Dir \left( p_{0}, \dots, p_{N}; \alpha_{0} + n_0, \dots, \alpha_{N} + n_N \right)
\\
P(p_i | \mathcal{D} ) & = 
\int dp_0 \dots dp_{i-1}dp_{i+i} \dots dp_N P(\mathcal{H} | \mathcal{D}) 
=
Beta( p_i; \alpha_i + n_i, \sum_{k=0}^{N} (\alpha_k + n_k) - \alpha_i - n_i )
\end{split}
$$

In [ ]:
def initial_params_dir(N):
    return np.ones(N)

def posterior_params_dir(data, initial_pars):
    u, c = np.unique(data, return_counts=True)
    post_pars = np.copy(initial_pars)
    for k, v in zip(u, c):
        post_pars[k] = post_pars[k] + v
    return post_pars

def posterior_dist_dir(params):
    return stats.dirichlet(alpha=params)

def posterior_nords_dir_rvs(params, nsamp):
    nords = np.empty(nsamp)
    d = posterior_dist_dir(params)
    probs = d.rvs(size=nsamp)
    for i, p in enumerate(probs):
        nords[i] = np.argmax(stats.multinomial.rvs(n=1, p=p))
    return nords

def marginal_pi_dist_dir(i, params):
    return stats.beta(a=params[i], b=np.sum(params) - params[i])

def posterior_pi_mean_95pdi(i, params):
    p = marginal_pi_dist_dir(i, params)
    m = p.mean()
    lower = p.ppf(0.025)
    upper = p.ppf(0.975)
    return m, lower, upper

Nmax = 30
s = 1.5
nsample = 1000

Npars = Nmax + 1
exact_dist = stats.zipfian(a=s, n=Npars, loc=-1)
data = exact_dist.rvs(nsample)
pars = initial_params_dir(Npars)
pars = posterior_params_dir(data, pars)
post_samp = posterior_nords_dir_rvs(pars, 100000)
pi = [posterior_pi_mean_95pdi(i, pars) for i in range(Npars)]

x = np.arange(0, Npars+1)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pmf(x), name='Точное распределение Ципфа', 
                         line_color='black'))
fig.add_trace(go.Histogram(x=data, histnorm='probability', name='Выборка', nbinsx=round(Nmax*2),
                         marker_color='black'))
fig.add_trace(go.Histogram(x=post_samp, histnorm='probability', name='$\mbox{Апостериорные } n_i$', 
                         marker_color='black', opacity=0.2, nbinsx=round(Nmax*2)))
fig.add_trace(go.Scatter(x=x, 
                         y=[p[0] for p in pi],
                         error_y=dict(type='data', symmetric=False, array=[p[2] - p[0] for p in pi], arrayminus=[p[0] - p[1] for p in pi]), 
                         name='$\mbox{Оценки } p_i$',
                         mode='markers',
                         line_color='red',
                         opacity=0.8))
fig.update_layout(title='Заказы на посетителя',
                  xaxis_title='$Заказы$',
                  yaxis_title='Вероятность',
                  xaxis_range=[-1, Nmax+1],
                  hovermode="x",
                  barmode="group",
                  height=550)
fig.show()
#fig.write_image("./figs/ch6_postdist.png", scale=2)
#Распределение заказов на посетителя. Точные конверсии в i заказов лежат внутри оцененных интервалов. 

In [ ]:
def posterior_nords_mean_rvs(params, nsample):
    ns = np.arange(len(params))
    probs = stats.dirichlet.rvs(alpha=params, size=nsample)
    means = np.sum(ns * probs, axis=1)
    return means

def prob_pb_gt_pa_samples(post_samp_A, post_samp_B):
    if len(post_samp_A) != len(post_samp_B):
        return None
    b_gt_a = np.sum(post_samp_B > post_samp_A)
    return b_gt_a / len(post_samp_A)

nsample = 3000
Nmax = 30
Npars = Nmax + 1

post_samp_len = 100000
A, B = {}, {}
s = 1.5
A['dist_pars'] = {'s': s}
B['dist_pars'] = {'s': s * 0.95}
for g in [A, B]:
    g['exact_dist'] = stats.zipfian(a=g['dist_pars']['s'], n=Npars, loc=-1)
    g['data'] = g['exact_dist'].rvs(nsample)
    g['post_pars'] = initial_params_dir(Npars)
    g['post_pars'] = posterior_params_dir(g['data'], g['post_pars'])
    g['post_nords'] = posterior_nords_dir_rvs(g['post_pars'], post_samp_len)
    g['post_means'] = posterior_nords_mean_rvs(g['post_pars'], post_samp_len)
    g['pi'] = [posterior_pi_mean_95pdi(i, g['post_pars']) for i in range(Npars)]

x = np.arange(0, Npars)
fig = go.Figure()
fig.add_trace(go.Bar(x=x, y=A['exact_dist'].pmf(x), name='Точное распределение A',
                        marker_color='black', opacity=0.2))
fig.add_trace(go.Bar(x=x, y=B['exact_dist'].pmf(x), name='Точное распределение Б',
                        marker_color='black', opacity=0.8))
fig.add_trace(go.Scatter(x=[A['exact_dist'].mean(), A['exact_dist'].mean()], 
                         y=[0, np.max(A['exact_dist'].pmf(x))*1.1],
                         name='Точное среднее A', 
                         mode='lines', line_dash='dash',
                         line_color='black', opacity=0.3))
fig.add_trace(go.Scatter(x=[B['exact_dist'].mean(), B['exact_dist'].mean()], 
                         y=[0, np.max(B['exact_dist'].pmf(x))*1.1],
                         name='Точное среднее Б', 
                         mode='lines', line_dash='dash',
                         line_color='black'))
fig.add_trace(go.Scatter(x=x - 0.1, 
                         y=[p[0] for p in A['pi']],
                         error_y=dict(type='data', symmetric=False, array=[p[2] - p[0] for p in A['pi']], arrayminus=[p[0] - p[1] for p in A['pi']]), 
                         name='$p_i, \mbox{ А}$',
                         line_color='black', opacity=0.3,
                         mode='markers'
                    ))
fig.add_trace(go.Scatter(x=x + 0.1, 
                         y=[p[0] for p in B['pi']],
                         error_y=dict(type='data', symmetric=False, array=[p[2] - p[0] for p in B['pi']], arrayminus=[p[0] - p[1] for p in B['pi']]), 
                         name='$p_i, \mbox{ Б}$',
                         line_color='black',
                         mode='markers'))
fig.update_layout(title='Заказы на посетителя',
                  xaxis_title='$Заказы$',
                  yaxis_title='Вероятность',
                  xaxis_range=[-1, Npars+1-20],
                  hovermode="x",
                  barmode="group",
                  height=550)
fig.show()
#fig.write_image("./figs/ch6_cmp_orig.png", scale=2)
#Точные распределения, точные средние количества заказов и оценки конверсий.

x = np.arange(0, Npars)
fig = go.Figure()
fig.add_trace(go.Scatter(x=[A['exact_dist'].mean(), A['exact_dist'].mean()], 
                         y=[0, np.max(A['exact_dist'].pmf(x))*1.1],
                         name='Точное среднее A', 
                         mode='lines', line_dash='dash',
                         line_color='black', opacity=0.3))
fig.add_trace(go.Scatter(x=[B['exact_dist'].mean(), B['exact_dist'].mean()], 
                         y=[0, np.max(B['exact_dist'].pmf(x))*1.1],
                         name='Точное среднее Б', 
                         mode='lines', line_dash='dash',
                         line_color='black'))
fig.add_trace(go.Histogram(x=A['post_means'], histnorm='probability', name='$E[n], \mbox{ А}$', 
                           marker_color='black', opacity=0.3, nbinsx=round(Nmax*2)))
fig.add_trace(go.Histogram(x=B['post_means'], histnorm='probability', name='$E[n], \mbox{ Б}$', 
                           marker_color='black', nbinsx=round(Nmax*2)))
fig.update_layout(title='Среднее количество заказов',
                  xaxis_title='$Заказы$',
                  yaxis_title='Вероятность',
                  xaxis_range=[-1, Npars+1-20],
                  hovermode="x",
                  barmode="group",
                  height=550)
fig.show()
#fig.write_image("./figs/ch6_cmp_means.png", scale=2)
#Оценки среднего количества заказов. Среднее Б выше А с вероятностью 90%.

print(f"P(E[n]_B > E[n]_A): {prob_pb_gt_pa_samples(A['post_means'], B['post_means'])}")

In [ ]:
cmp = pd.DataFrame(columns=['A', 'B', 'best_exact', 'exp_samp_size', 'A_exp', 'B_exp', 'best_exp', 'p_best'])

s = 1.5
Nmax = 30
Npars = Nmax + 1
nexps = 100
cmp['A'] = [s] * nexps
cmp['B'] = s * (1 + stats.uniform.rvs(loc=-0.05, scale=0.1, size=nexps))

n_samp_max = 200000
n_samp_step = 5000

prob_stop = 0.95
for i in range(nexps):
    s_a = cmp.at[i, 'A']
    s_b = cmp.at[i, 'B']
    exact_dist_a = stats.zipfian(a=s_a, n=Npars, loc=-1)
    exact_dist_b = stats.zipfian(a=s_b, n=Npars, loc=-1)
    cmp.at[i, 'best_exact'] = 'A' if exact_dist_a.mean() > exact_dist_b.mean() else 'B'
    n_samp_total = 0
    pars_a = initial_params_dir(Npars)
    pars_b = initial_params_dir(Npars)
    while n_samp_total < n_samp_max:
        data_a = exact_dist_a.rvs(n_samp_step)
        data_b = exact_dist_b.rvs(n_samp_step)
        n_samp_total += n_samp_step
        pars_a = posterior_params_dir(data_a, pars_a)
        pars_b = posterior_params_dir(data_b, pars_b)
        post_samp_len = 10000
        post_means_a = posterior_nords_mean_rvs(pars_a, post_samp_len)
        post_means_b = posterior_nords_mean_rvs(pars_b, post_samp_len)
        pb_gt_pa = prob_pb_gt_pa_samples(post_means_a, post_means_b)
        best_gr = 'B' if pb_gt_pa >= prob_stop else 'A' if (1 - pb_gt_pa) >= prob_stop else None
        if best_gr:
            cmp.at[i, 'A_exp'] = post_means_a.mean()
            cmp.at[i, 'B_exp'] = post_means_b.mean()
            cmp.at[i, 'exp_samp_size'] = n_samp_total
            cmp.at[i, 'best_exp'] = best_gr
            cmp.at[i, 'p_best'] = pb_gt_pa
            break
    print(f'done {i}: nsamp {n_samp_total}, best_gr {best_gr}, P(B>A) {pb_gt_pa}')

cmp['correct'] = cmp['best_exact'] == cmp['best_exp']
display(cmp.head(10))
cor_guess = np.sum(cmp['correct'])
print(f"Nexp: {nexps}, Correct Guesses: {cor_guess}, Accuracy: {cor_guess / nexps}")

$$
\begin{split}
P(\mathcal{D} | \mathcal{H}) & = Mult(n_0, n_1, n_2 | p_0, p_1, p_2) = \frac{(n_0 + n_1 + n_2)!}{n_{0}! n_{1}! n_{2}!} p_{0}^{n_{0}} p_{1}^{n_{1}} p_{2}^{n_{2}} 
\\
P(\mathcal{H}) & = 
Dir \left( p_{0}, \dots, p_{N}; \alpha_{0}, \dots, \alpha_{N} \right) = 
\dfrac{1}{B( \alpha_{0}, \dots, \alpha_{N} )} \prod_{i=0}^{N} p_{i}^{\alpha_{i}-1},
\qquad
\sum_{i=0}^{N} p_i = 1,
\qquad
p_i \in [0, 1], 
\qquad
B(\alpha_{0}, \dots, \alpha_{N}) = 
\frac{\prod \limits_{i=0}^{N} \Gamma( \alpha_{i} )}
{\Gamma \left( \sum \limits_{i=0}^{N} \alpha_{i} \right)}
\\
P(\mathcal{H} | \mathcal{D}) 
& \propto Mult(n_0, \dots, n_N | p_0, \dots, p_N) Dir \left( p_{0}, \dots, p_{N}; \alpha_{0}, \dots, \alpha_{N} \right)
\\
& \propto
p_{0}^{n_{0}} \dots p_{N}^{n_{N}} 
\prod _{i=0}^{N} p_{i}^{\alpha_{i}-1}
\\
& \propto
\prod_{i=0}^{N} p_{i}^{n_{i} + \alpha_{i} - 1}
\\
& =
Dir \left( p_{0}, \dots, p_{N}; \alpha_{0} + n_0, \dots, \alpha_{N} + n_N \right)
\\
P(p_i | \mathcal{D} ) & = 
\int dp_0 \dots dp_{i-1}dp_{i+i} \dots dp_N P(\mathcal{H} | \mathcal{D}) 
=
Beta( p_i; \alpha_i + n_i, \sum_{k=0}^{N} (\alpha_k + n_k) - \alpha_i - n_i )
\end{split}
$$

$$
P(\mathcal{H} | \mathcal{D}) \propto P(\mathcal{D} | \mathcal{H}) P(\mathcal{H})
$$

$$
P(\mathcal{D} | \mathcal{H}) = P(n_s, N | p) = \mbox{Binom}(n_s, N | p) = C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
$$

$$
P(\mathcal{H}) = P(p) = \mbox{Beta}(p; \alpha, \beta) = 
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)} p^{\alpha-1}(1-p)^{\beta-1}
$$

$$
\begin{split}
P(\mathcal{H} | \mathcal{D}) & = P(p | n_s, N) 
\\
& \propto \mbox{Binom}(n_s, N | p) \mbox{Beta}(p; \alpha, \beta)
\\
& \propto C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)} p^{\alpha-1}(1-p)^{\beta-1}
\\
& \propto p^{n_s + \alpha - 1} (1-p)^{N - n_s + \beta - 1}
\\
& = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)
\end{split}
$$

p - вероятность продления, $p \leftarrow 1-\theta$      
s - продления, $s \leftarrow k$    
$n_0$ - пользователи на начальном этапе $n_0 \leftarrow r$

$$
P(s; n_0, p) = \binom{s+n_0-1}{s} (1-p)^{n_0} p^s
$$


Вероятность $p$ можно подбирать по данным.  
Сопряженным априорным будет бета-распределение.  

$$
P(\mathcal{H} | \mathcal{D}) \propto P(\mathcal{D} | \mathcal{H}) P(\mathcal{H})
$$

$$
P(\mathcal{D} | \mathcal{H}) = P(f, s | p) = \mbox{NegBinom}(s, n_0 | p) = C_{s+n_0-1}^{s} (1-p)^{n_0}  p^{s}
$$

$$
P(\mathcal{H}) = P(p) = \mbox{Beta}(p; \alpha, \beta) = 
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)} p^{\alpha-1}(1-p)^{\beta-1}
$$

$$
\begin{split}
P(\mathcal{H} | \mathcal{D}) & = P(p | s, n_0) 
\\
& \propto \mbox{NegBinom}(s, n_0 | p) \mbox{Beta}(p; \alpha, \beta)
\\
& \propto C_{s+n_0-1}^{s} (1-p)^{n_0}  p^{s}
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)} p^{\alpha-1}(1-p)^{\beta-1}
\\
& \propto p^{s + \alpha - 1} (1-p)^{n_0 + \beta - 1}
\\
& = \mbox{Beta}(p; \alpha + s, \beta + n_0)
\end{split}
$$ 

Не хватает продлений на след. шагах?  
Все войдут в s ?  

Нельзя ставить негативное биномиальное как правдоподобие.  
Должны пройти все продления.  
Нужно ограничение на число шагов. 